[Reference1](https://towardsdatascience.com/automate-email-with-python-1e755d9c6276) <br>
[Reference2](https://towardsdatascience.com/notify-with-python-41b77d51657e)

# MIME

```python
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.mime.audio import MIMEAudio

msg = MIMEMultipart  # initialize the MIME object, containing our email message
msg.attach(MIMEText(text))  # add text contents

img_data = open('image.jpg', 'rb').read()  # read an image binary data
msg.attach(MIMEImage(img_data))  # add the image to our message object

# read in attachment as binary
with open('report.docx', 'rb') as f:
    file = MIMEApplication(f.read())  # read the attachment file
msg.attach(file)  # add the attachment to our message object
```

```python
import os
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart

def message(subject="Python Notification", text="", img=None, attachment=None):
    # build message contents
    msg = MIMEMultipart()
    msg['Subject'] = subject  # add in the subject
    msg.attach(MIMEText(text))  # add text contents

    # check if we have anything given in the img parameter
    if img is not None:
        # if we do, we want to iterate through the images, so let's check that
        # what we have is actually a list
        if type(img) is not list:
            img = [img]  # if it isn't a list, make it one
        # now iterate through our list
        for one_img in img:
            img_data = open(one_img, 'rb').read()  # read the image binary data
            # attach the image data to MIMEMultipart using MIMEImage, we add
            # the given filename use os.basename
            msg.attach(MIMEImage(img_data, name=os.path.basename(one_img)))

    # we do the same for attachments as we did for images
    if attachment is not None:
        if type(attachment) is not list:
            attachment = [attachment]  # if it isn't a list, make it one
            
        for one_attachment in attachment:
            with open(one_attachment, 'rb') as f:
                # read in the attachment using MIMEApplication
                file = MIMEApplication(
                    f.read(),
                    name=os.path.basename(one_attachment)
                )
            # here we edit the attached file metadata
            file['Content-Disposition'] = f'attachment; filename="{os.path.basename(one_attachment)}"'
            msg.attach(file)  # finally, add the attachment to our message object
    return msg
```

```python
email_msg = message(
    text="Model processing complete, please see attached data.",
    img=['accuracy.png', 'loss.png'],
    attachments=['data_in.csv', 'data_out.csv']
)
```

# SMTP

![smtp](https://miro.medium.com/max/1400/1*tGCyXhXldhyxxGUsh9Yw6w.png)

```python
import smtplib

# initialize connection to our email server, we will use Outlook here
smtp = smtplib.SMTP('smtp-mail.outlook.com', port='587')

smtp.ehlo()  # send the extended hello to our server
smtp.starttls()  # tell server we want to communicate with TLS encryption

smtp.login('joe.bloggs@outlook.com', 'Password123')  # login to our email server

# send our email message 'msg' to our boss
smtp.sendmail('joe.bloggs@outlook.com',
              'joes.boss@outlook.com',
              msg.as_string())
              
smtp.quit()  # finally, don't forget to close the connection
```

# Notify

```python
import notify

START = datetime.now()  # this line would be placed before model training begins
MODELNAME = "Synthwave GAN"  # giving us our model name
NOTIFY = 100  # so we send an update notification every 100 epochs

# for each epoch e, we would include the following code
if e % notify_epoch == 0 and e != 0:
    # here we create the email body message
    txt = (f"{MODELNAME} update as of "
           f"{datetime.now().strftime('%H:%M:%S')}.")

    # we build the MIME message object with notify.message
    msg = notify.message(
        subject='Synthwave GAN',
        text=txt,
        img=[
            f'../visuals/{MODELNAME}/epoch_{e}_loss.png',
            f'../visuals/{MODELNAME}/epoch_{e}_iter_{i}.png'
        ]
    )  # note that we attach two images here, the loss plot and
    #    ...a generated image output from our model
           
    notify.send(msg)  # we then send the message
```

```python
import os
import notify
from data import Sql  # see https://jamescalam.github.io/pysqlplus/lib/data/sql.html

dt = Sql('database123', 'server001')  # setup the connection to SQL Server

for i, file in enumerate(os.listdir('../data/new')):
    dt.push_raw(f'../data/new/{file}')  # push a file to SQL Server

# once the upload is complete, send a notification
# first we create the message
msg = notify.message(
    subject='SQL Data Upload',
    text=f'Data upload complete, {i} files uploaded.',
)

# send the message
notify.send(msg)
```

```python
end = datetime.datetime.now()  # get the ending datetime

# get the total runtime in hours:minutes:seconds
hours, rem = divmod((end - start).seconds, 3600)
mins, secs = divmod(rem, 60)
runtime = '{:02d}:{:02d}:{:02d}'.format(hours, mins, secs)

# now built our message
notify.msg(
    subject="Cashflow Model Completion",
    text=(f'{len(model.output)} loans processed.\n'
          f'Total runtime: {runtime}'),
    img=[
        '../vis/loan01_amortisation.png',
        '../vis/loan07_amortisation.png',
        '../vis/loan01_profit_and_loss.png',
        '../vis/loan07_profit_and_loss.png'
    ]
)

notify.send(msg)  # and send it
```

```python
import smtplib
import socket

def send(msg, server='smtp-mail.outlook.com', port='587'):
    # contain following in try-except in case of momentary network errors
    try:
        # initialise connection to email server, the default is Outlook
        smtp = smtplib.SMTP(server, port)
        # this is the 'Extended Hello' command, essentially greeting our SMTP or ESMTP server
        smtp.ehlo()
        # this is the 'Start Transport Layer Security' command, tells the server we will
        # be communicating with TLS encryption
        smtp.starttls()
        
        # read email and password from file
        with open('../data/email.txt', 'r') as fp:
            email = fp.read()
        with open('../data/password.txt', 'r') as fp:
            pwd = fp.read()
            
        # login to outlook server
        smtp.login(email, pwd)
        # send notification to self
        smtp.sendmail(email, email, msg.as_string())
        # disconnect from the server
        smtp.quit()
    except socket.gaierror:
        print("Network connection error, email not sent.")
```

```python
# build a message object
msg = message(text="See attached!", img='important.png',
              attachment='data.csv')send(msg)  # send the email (defaults to Outlook)
```